In [23]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Activation
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2

In [6]:
# Getting mask folders with mask and without mask from current directory

mask_folder="./with_mask"
no_mask_folder="./without_mask"

In [7]:
img_data=[]
labels=[]
def load_imgdata(mask_folder,no_mask_folder):
    for folder,label in [(mask_folder,1),(no_mask_folder,0)]:
        for img_name in os.listdir(folder):
            img_path=os.path.join(folder,img_name)
            image=load_img(img_path,target_size=(100,100))
            image_array=img_to_array(image)
            img_data.append(image_array)
            labels.append(label)
    return np.array(img_data),np.array(labels)

In [9]:
img_data,label=load_imgdata(mask_folder,no_mask_folder)

C:\Anaconda3\Lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [10]:
img_data=img_data/255.0

In [16]:
x_train,x_test,y_train,y_test=train_test_split(img_data,label,test_size=0.3,random_state=42)
print(f"Training Samples : {len(x_train)} Testing Samples : {len(x_test)}")

Training Samples : 2692 Testing Samples : 1154


In [19]:
base_model=MobileNetV2(input_shape=(100,100,3),include_top=False,weights="imagenet")

C:\Users\Admin\AppData\Local\Temp\ipykernel_15172\1471432744.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model=MobileNetV2(input_shape=(100,100,3),include_top=False,weights="imagenet")


In [20]:
base_model.trainable=False

In [21]:
model=Sequential([
                  base_model,
                  Flatten(),
                  Dense(128,activation='relu'),
                  Dropout(0.3),
                  Dense(1,activation='sigmoid')])

In [22]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
datagen=ImageDataGenerator(
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest'
 )

In [26]:
model.fit(datagen.flow(x_train,y_train,batch_size=32),epochs=10,validation_data=(x_test,y_test))

C:\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 28s 257ms/step - accuracy: 0.8425 - loss: 1.8785 - val_accuracy: 0.9827 - val_loss: 0.0648
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 236ms/step - accuracy: 0.9704 - loss: 0.1032 - val_accuracy: 0.9870 - val_loss: 0.0549
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 232ms/step - accuracy: 0.9720 - loss: 0.0721 - val_accuracy: 0.9818 - val_loss: 0.0504
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 235ms/step - accuracy: 0.9658 - loss: 0.1107 - val_accuracy: 0.9913 - val_loss: 0.0325
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 235ms/step - accuracy: 0.9781 - loss: 0.0674 - val_accuracy: 0.9931 - val_loss: 0.0311
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 232ms/step - accuracy: 0.9660 - loss: 0.0949 - val_accuracy: 0.9818 - val_loss: 0.0660
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 232ms/step - accuracy: 0.9756 - loss: 0.0594 - val_accuracy: 0.9896 - val_loss: 0.0387
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 236ms/step - accuracy: 0.9688 - loss: 0.0872 - val_accu

In [27]:
loss,accuracy=model.evaluate(x_test,y_test)
print(f"Accuracy : {accuracy*100:.2f}%")

37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.9905 - loss: 0.0548
Accuracy : 99.13%


In [28]:
model.save("mask.h5")